In [2]:
with open('input') as f:
    content = f.read()

In [3]:
# Regex approach for part 1
import re

matcher = re.compile(r'mul\((\d+),(\d+)\)')

matches = matcher.findall(content)

result = 0
for a, b in matches:
    result += int(a) * int(b)

print(result)

174960292


In [4]:
# Regex approach for part 2
import re

matcher = re.compile(r"(mul\((\d+),(\d+)\)|do\(\)|don't\(\))")

matches = matcher.findall(content)

result = 0
enabled = True
for m, a, b in matches:
    if m == 'do()':
        enabled = True
    elif m == "don't()":
        enabled = False
    else:
        assert m == f'mul({a},{b})'
        if enabled:
            result += int(a) * int(b)

print(result)

56275602


In [5]:
# Custom parser approach for part 1 and part 2

from dataclasses import dataclass
from typing import Protocol

@dataclass(frozen=True)
class Token:
    start: int
    end: int

@dataclass(frozen=True)
class NumberToken(Token):
    value: int
    

@dataclass(frozen=True)
class MultiplyToken(Token):
    pass

@dataclass(frozen=True)
class OpenParenToken(Token):
    pass

@dataclass(frozen=True)
class CloseParenToken(Token):
    pass

@dataclass(frozen=True)
class CommaToken(Token):
    pass

@dataclass(frozen=True)
class DoToken(Token):
    pass

@dataclass(frozen=True)
class DontToken(Token):
    pass

@dataclass(frozen=True)
class UnknownToken(Token):
    pass


TokenUnion = NumberToken | MultiplyToken | OpenParenToken | CloseParenToken | CommaToken | UnknownToken | DoToken | DontToken

# Tokenization
tokens: list[TokenUnion] = []

i = 0
while i < len(content):
    c = content[i]

    if c == '(':
        tokens.append(OpenParenToken(start=i, end=i+1))
        i += 1
    elif c == ')':
        tokens.append(CloseParenToken(start=i, end=i+1))
        i += 1
    elif c.isalpha():
        if content[i:i+5] == "don't":
            # print('found a dont')
            tokens.append(DontToken(start=i, end=i+5))
            i += 5
        elif content[i:i+3] == 'mul':
            # print('found a mul')
            tokens.append(MultiplyToken(start=i, end=i+3))
            i += 3
        elif content[i:i+2] == 'do':
            # print('found a do')
            tokens.append(DoToken(start=i, end=i+2))
            i += 2
        else:
            tokens.append(UnknownToken(start=i, end=i+1))
            i += 1
    elif c.isdigit():
        j = i
        for k in content[i:]:
            if not k.isdigit():
                break
            j += 1
        slice = content[i:j]
        if len(slice.lstrip('0')) != len(slice):
            tokens.append(UnknownToken(start=i, end=j))
            i += 1
        else:
            tokens.append(NumberToken(value=int(slice), start=i, end=j))
            i = j
    elif c == ',':
        tokens.append(CommaToken(start=i, end=i+1))
        i += 1
    else:
        tokens.append(UnknownToken(start=i, end=i+1))
        i += 1

class Expression(Protocol):
    @property
    def result(self) -> int:
        ...

class Statement(Protocol):
    ...

@dataclass(frozen=True)
class Do(Statement):
    pass

@dataclass(frozen=True) 
class Dont(Statement):
    pass


@dataclass(frozen=True)
class Multiplication(Expression):
    a: int
    b: int

    @property
    def result(self):
        return self.a * self.b

# Parsing (part 1)
mult_definition: list[type] = [MultiplyToken, OpenParenToken, NumberToken, CommaToken, NumberToken, CloseParenToken]
mults: list[Multiplication] = []
i = 0
while i < len(tokens):
    if all(isinstance(tokens[i+j], mult_definition[j]) for j in range(len(mult_definition))):
        lhs = tokens[i+2]
        rhs = tokens[i+4]

        assert isinstance(lhs, NumberToken)
        assert isinstance(rhs, NumberToken)

        mults.append(Multiplication(lhs.value, rhs.value))
        i += len(mult_definition)
    else:
        i += 1

# Evaluation
result = 0
for mult in mults:
    result += mult.result

if (len(mults) > 0):
    print(result)
else:
    print('No multiplications found')

# Parsing (part 2)
do_definition: list[type] = [DoToken, OpenParenToken, CloseParenToken]
dont_definition: list[type] = [DontToken, OpenParenToken, CloseParenToken]
grammar: list[list[type]] = [
    do_definition,
    dont_definition,
    mult_definition,
]
statements: list[Do | Dont | Multiplication] = []
i = 0
while i < len(tokens):
    matched = False
    for definition in grammar:
        if all(isinstance(tokens[i+j], definition[j]) for j in range(len(definition))):
            if definition == do_definition:
                statements.append(Do())
            elif definition == dont_definition:
                statements.append(Dont())
            elif definition == mult_definition:
                lhs = tokens[i+2]
                rhs = tokens[i+4]

                assert isinstance(lhs, NumberToken)
                assert isinstance(rhs, NumberToken)

                statements.append(Multiplication(lhs.value, rhs.value))
            i += len(definition)
            matched = True
            break
            
    if not matched:
        i += 1

# Evaluation
enabled = True
result = 0
for statement in statements:
    if isinstance(statement, Multiplication) and enabled:
        result += statement.result
    elif isinstance(statement, Do):
        enabled = True
    elif isinstance(statement, Dont):
        enabled = False

print(result)


174960292
56275602
